In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import csv
from datetime import datetime, timedelta
from dateutil.relativedelta import *
from sklearn import preprocessing
import numpy as np
from scipy import sparse
from sklearn.cross_validation import KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn import naive_bayes
import pandas as pd
import xgboost as xgb
import time

In [2]:
dtypes = {"Date": datetime, "StateHoliday": np.dtype(str), "SchoolHoliday": np.dtype(int)}

# The read_csv returns an error when reading the stores data because of missing values but
# works when I don't specify the dtypes
store_dtypes = {"CompetitionSinceYear": np.dtype(int), "CompetitionSinceMonth": np.dtype(int), 
                "Promo2SinceYear": np.dtype(int), "Promo2SinceWeek": np.dtype(int)}
data = pd.read_csv('data/train.csv', dtype=dtypes, parse_dates=[2])
stores = pd.read_csv('data/store.csv')

test = pd.read_csv('data/test.csv', dtype=dtypes, parse_dates=[3])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Date             1017209 non-null datetime64[ns]
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 77.6+ MB


In [4]:
# remove rows with sales of 0
data = data[data.Sales > 0]

# extract sales to separate Series
# sales = data[['Store','Sales']]

# remove Sales and Customers columns
# data.drop(['Sales','Customers'], axis=1, inplace=True)

# check Open column and set to open if NaN
data.Open.fillna(1, inplace=True)

In [5]:
# Check all the values for StateHoliday
pd.unique(data.StateHoliday.ravel())

array(['0', 'a', 'b', 'c'], dtype=object)

In [6]:
# Check all the values for SchoolHoliday
pd.unique(data.SchoolHoliday.ravel())

array([1, 0])

In [7]:
# recode StateHoliday as dummy variable
def stateHoliday_asDummy(df):
    sh = pd.get_dummies(df['StateHoliday'])

    sh.rename(columns={'0':'noHoliday','a':'PublicHoliday'}, inplace=True)
    sh[['noHoliday','PublicHoliday']] = sh[['noHoliday','PublicHoliday']].astype(int)
    try:
        sh.rename(columns={'b':'EasterHoliday','c':'XmasHoliday'}, inplace=True)
        sh[['EasterHoliday','XmasHoliday']] = sh[['EasterHoliday','XmasHoliday']].astype(int)
    except:
        pass

    df = pd.concat([df,sh], axis=1)
    return df

In [8]:
# recode SchoolHoliday as binary instead of categorical
def schoolHoliday_asDummy(df):
    df.SchoolHoliday = df.SchoolHoliday.apply(int)
    return df

In [9]:
def recodeCategorical_asInt(df, fn, newfn):
    df[newfn] = pd.Categorical.from_array(df[fn]).codes
    return df

In [10]:
# Dummy variable approach
data = stateHoliday_asDummy(data)
data = schoolHoliday_asDummy(data)

# Categorical variable approach
data = recodeCategorical_asInt(data, 'SchoolHoliday', 'SchoolHolidayVal')
data = recodeCategorical_asInt(data, 'StateHoliday', 'StateHolidayVal')

In [11]:
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,noHoliday,PublicHoliday,EasterHoliday,XmasHoliday,SchoolHolidayVal,StateHolidayVal
0,1,5,2015-07-31,5263,555,1,1,0,1,1,0,0,0,1,0
1,2,5,2015-07-31,6064,625,1,1,0,1,1,0,0,0,1,0
2,3,5,2015-07-31,8314,821,1,1,0,1,1,0,0,0,1,0
3,4,5,2015-07-31,13995,1498,1,1,0,1,1,0,0,0,1,0
4,5,5,2015-07-31,4822,559,1,1,0,1,1,0,0,0,1,0


In [12]:
# find NaN values in test data
#test.isnull().sum().sum()  # indicates 11 missing values
# test.Open.isnull().sum()  # they are all in the Open column
test.Open.fillna(1, inplace=True)
test.Open = test.Open.astype(int)

In [13]:
test = stateHoliday_asDummy(test)
test = schoolHoliday_asDummy(test)

test = recodeCategorical_asInt(test, 'StateHoliday', 'StateHolidayVal')
test = recodeCategorical_asInt(test, 'SchoolHoliday', 'SchoolHolidayVal')

In [14]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,noHoliday,PublicHoliday,StateHolidayVal,SchoolHolidayVal
0,1,1,4,2015-09-17,1,1,0,0,1,0,0,0
1,2,3,4,2015-09-17,1,1,0,0,1,0,0,0
2,3,7,4,2015-09-17,1,1,0,0,1,0,0,0
3,4,8,4,2015-09-17,1,1,0,0,1,0,0,0
4,5,9,4,2015-09-17,1,1,0,0,1,0,0,0


In [15]:
# Take a look at the store metadata
stores.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,NaN,NaN,NaN
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN


In [16]:
# list unique StoreType values
pd.unique(stores.StoreType.ravel())

array(['c', 'a', 'd', 'b'], dtype=object)

In [17]:
# list unique Assortment values
pd.unique(stores.Assortment.ravel())

array(['a', 'c', 'b'], dtype=object)

In [18]:
# recode StoreType and Assortment as dummy variables
def storeType_asDummy(df):
    st = pd.get_dummies(df['StoreType'])    
    st.rename(columns={'a':'StoreTypeA','b':'StoreTypeB','c':'StoreTypeC','d':'StoreTypeD'}, inplace=True)
    df = pd.concat([df,st], axis=1)    
    df[['StoreTypeA','StoreTypeB','StoreTypeC','StoreTypeD']] = df[['StoreTypeA','StoreTypeB','StoreTypeC','StoreTypeD']].astype(int)
    return df
    
def assortment_asDummy(df):
    ass = pd.get_dummies(df['Assortment'])
    ass.rename(columns={'a':'BasicAssortment','b':'ExtraAssortment','c':'ExtendedAssortment'}, inplace=True)
    df = pd.concat([df,ass], axis=1)
    df[['BasicAssortment','ExtraAssortment','ExtendedAssortment']] = df[['BasicAssortment','ExtraAssortment','ExtendedAssortment']].astype(int)
    return df

In [19]:
# Convert PromoInterval to dummy variable
def promoInterval_asDummy(df):
    pi = pd.get_dummies(df['PromoInterval'])
    pi.rename(columns={'Feb,May,Aug,Nov':'PromoIntFebMayAugNov','Jan,Apr,Jul,Oct':'PromoIntJanAprJulOct',
                       'Mar,Jun,Sept,Dec':'PromoIntMarJunSeptDec','None':'PromoIntNone'}, inplace=True)
    df = pd.concat([df,pi], axis=1)
    df[['PromoIntFebMayAugNov','PromoIntJanAprJulOct','PromoIntMarJunSeptDec','PromoIntNone']] = \
        df[['PromoIntFebMayAugNov','PromoIntJanAprJulOct','PromoIntMarJunSeptDec','PromoIntNone']].astype(int)
    return df

In [20]:
# Clean up NaN values

# Set competition distance to 0 if NaN
stores.CompetitionDistance.fillna(0, inplace=True)

# Set CompetitionOpenSince values to 0 if NaN
stores.CompetitionOpenSinceMonth.fillna(0, inplace=True)
stores.CompetitionOpenSinceYear.fillna(0, inplace=True)

# Set Promo2Since values to 0 if NaN
stores.Promo2SinceWeek.fillna(0, inplace=True)
stores.Promo2SinceYear.fillna(0, inplace=True)

# Set PromoInterval value to the string 'None' if NaN
stores.PromoInterval.fillna('None', inplace=True)

In [21]:
#Dummy variables
stores = storeType_asDummy(stores)
stores = assortment_asDummy(stores)
stores = promoInterval_asDummy(stores)

In [22]:
# Categorical variables
stores = recodeCategorical_asInt(stores, 'StoreType', 'StoreTypeVal')
stores = recodeCategorical_asInt(stores, 'Assortment', 'AssortmentVal')
stores = recodeCategorical_asInt(stores, 'PromoInterval', 'PromoIntervalVal')

In [23]:
# perform a left outer join of the data and stores dataframes
all_data = pd.merge(data, stores, on='Store', how='left')

# same for the test data
all_test = pd.merge(test, stores, on='Store', how='left')

In [24]:
all_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 35 columns):
Id                           41088 non-null int64
Store                        41088 non-null int64
DayOfWeek                    41088 non-null int64
Date                         41088 non-null datetime64[ns]
Open                         41088 non-null int64
Promo                        41088 non-null int64
StateHoliday                 41088 non-null object
SchoolHoliday                41088 non-null int64
noHoliday                    41088 non-null int64
PublicHoliday                41088 non-null int64
StateHolidayVal              41088 non-null int8
SchoolHolidayVal             41088 non-null int8
StoreType                    41088 non-null object
Assortment                   41088 non-null object
CompetitionDistance          41088 non-null float64
CompetitionOpenSinceMonth    41088 non-null float64
CompetitionOpenSinceYear     41088 non-null float64
Promo2                   

In [25]:
# Now that we have the observation date combined with store metadata, we can recompute 
# the promosince... and competitionsince... year/week values as weeks before observation date

# This takes a really long time to execute...

def monthsPromo(x):
    months = 0
    if x.Promo2SinceYear > 0:
        d1s = str(int(x.Promo2SinceYear)) + '-' + str(int(x.Promo2SinceWeek))
        d1 = datetime.strptime(d1s + '-1', "%Y-%W-%w")
        months = (x.Date.year - d1.year) * 12 + x.Date.month - d1.month
        if months < 0:
            months = 0
    return months

def weeksPromo(x):
    weeks = 0
    if x.Promo2SinceYear > 0:
        d1s = str(int(x.Promo2SinceYear)) + '-' + str(int(x.Promo2SinceWeek))
        d1 = datetime.strptime(d1s + '-1', "%Y-%W-%w")
        m1 = (d1 - timedelta(days=d1.weekday()))
        m2 = (x.Date - timedelta(days=x.Date.weekday()))
        weeks = (m2 - m1).days / 7
        if weeks < 0:
            weeks = 0
    return weeks

def monthsComp(x):
    months = 0
    if x.CompetitionOpenSinceYear > 0:
        d1s = str(int(x.CompetitionOpenSinceYear)) + '-' + str(int(x.CompetitionOpenSinceMonth))
        d1 = datetime.strptime(d1s, "%Y-%m")
        r = relativedelta(x.Date,d1)
        months = r.years*12 + r.months
        if months < 0:
            months = 0
    return months

def calculateCompPromoFeatures(df):
    # Calculate the PromoWeeks and CompetitionMonths as single Series
    df['Promo2SinceWeeks'] = df.apply(weeksPromo, axis=1)
    df['Promo2SinceMonths'] = df.apply(monthsPromo, axis=1)
    df['CompetitionOpenSinceMonths'] = df.apply(monthsComp, axis=1)
    return df

In [26]:
all_data = calculateCompPromoFeatures(all_data)
all_test = calculateCompPromoFeatures(all_test)

In [154]:
all_data['day'] = all_data.Date.dt.day
all_data['month'] = all_data.Date.dt.month
all_data['year'] = all_data.Date.dt.year

all_test['day'] = all_test.Date.dt.day
all_test['month'] = all_test.Date.dt.month
all_test['year'] = all_test.Date.dt.year

In [29]:
# Calculate Day of Year from Date
#all_data['DayOfYear'] = all_data.Date.dt.dayofyear
#all_test['DayOfYear'] = all_test.Date.dt.dayofyear

In [30]:
# separate store data into individua=l dataframes
store_data = []
for store_id in all_data.Store.unique():
    store_data.append(all_data.loc[all_data.Store == store_id])

In [138]:
# Construct a feature list for the features we want to include in the models
feature_list_1 = ['Store','DayOfWeek','DayOfYear', 'CompetitionDistance', 'CompetitionOpenSinceMonths',
                'Promo', 'Promo2', 'Promo2SinceWeeks', 'noHoliday','PublicHoliday','EasterHoliday','XmasHoliday'
                'StoreTypeA', 'StoreTypeB', 'StoreTypeC', 'StoreTypeD', 'BasicAssortment', 'ExtraAssortment',
                'ExtendedAssortment', 'PromoIntNone', 'PromoIntFebMayAugNov', 'PromoIntJanAprJulOct',
                'PromoIntMarJunSeptDec']

feature_list_2 = ['Store','DayOfWeek','DayOfYear', 'CompetitionDistance', 'CompetitionOpenSinceMonths',
                'Promo', 'Promo2', 'Promo2SinceMonths', 'SchoolHolidayVal','StateHolidayVal',
                'StoreTypeVal', 'AssortmentVal', 'PromoIntervalVal']


feature_list_3 = ['Store','DayOfWeek','day', 'month', 'year', 'CompetitionDistance', 'CompetitionOpenSinceMonths',
                'Promo', 'Promo2', 'Promo2SinceMonths', 'SchoolHolidayVal','StateHolidayVal',
                'StoreTypeVal', 'AssortmentVal', 'PromoIntervalVal']

In [32]:
# from the xgboost source code it looks like the custom feval function is passed two parameters: y_hat and y
# where the y is a DMatrix object and the Y-hat the output of the prediction for that Y value.
# The return is supposed to be (in this case, I believe) a string, value tuple
# https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/core.py
# Also, how to use this function was not very clear but this example gives a clue:
# https://github.com/dmlc/xgboost/blob/master/demo/guide-python/cross_validation.py

def rmspe(y_hat, dmat):
    # we need to reverse the log(Y)
    y = np.exp(dmat.get_label())
    return "rmspe", np.sqrt(np.mean(((np.exp(y_hat)-y)/(y-1))**2))

In [172]:
separate_model_feature_list = ['DayOfWeek','day', 'month', 'year', 'CompetitionDistance', 'CompetitionOpenSinceMonths',
                'Promo', 'Promo2', 'Promo2SinceMonths', 'SchoolHolidayVal','StateHolidayVal',
                'StoreTypeVal', 'AssortmentVal', 'PromoIntervalVal']

def prepDataframe(df):
    # take the data from the all_data dataframe and sales dataframe
    X_train, X_test, Y_train, Y_test = train_test_split(df, df.Sales, test_size=0.10, random_state=42)
    X_train = X_train[separate_model_feature_list]
    X_test  = X_test[separate_model_feature_list]

    # take the log of the Y values
    Ylog_train = np.log(Y_train)
    Ylog_test  = np.log(Y_test)
    
    # scale the data
    scaler = preprocessing.StandardScaler().fit(X_train)
    scaled_trainX = scaler.transform(X_train)
    scaled_testX = scaler.transform(X_test, len(X_test))
    
    return(scaled_trainX, Ylog_train, scaled_testX, Ylog_test, scaler)

In [173]:

# Stopping. Best iteration:
# [546]	eval-rmspe:0.443694	train-rmspe:0.472175  (score 0.39930)
param_1 = {'max_depth':10, 'objective':'reg:linear', 'silent':1, 'eta': 0.5,
         'booster': 'gblinear', 'alpha' : 0.01, 'lambda' : 1}

# tree booster got to - 
# Stopping. Best iteration:
# [185]	eval-rmspe:0.111982	train-rmspe:0.016138 (Score of 0.13704)
param = {'max_depth':20, 'objective':'reg:linear', 'silent':1, 'eta': 0.2,
         'booster': 'gbtree', 'alpha' : 0.001, 'lambda' : 1, 'gamma': 0}

# [164]	eval-rmspe:0.115576	train-rmspe:0.012887  (Score of 0.1398)
param_3 = {'max_depth':20, 'objective':'reg:linear', 'silent':1, 'eta': 0.2,
         'booster': 'gbtree', 'alpha' : 0.001, 'lambda' : 0.5, 'gamma': 0}

# [60]	eval-rmspe:0.119277	train-rmspe:0.002058
param_4 = {'max_depth':50, 'objective':'reg:linear', 'silent':1, 'eta': 0.2,
         'booster': 'gbtree', 'alpha' : 0.0001, 'lambda' : 0.5, 'gamma': 0, 'eval_metric':'rmse'}

# [60]	eval-rmspe:0.119277	train-rmspe:0.002058
param_5 = {'max_depth':50, 'objective':'reg:linear', 'silent':1, 'eta': 0.2,
         'booster': 'gbtree', 'alpha' : 0.001, 'lambda' : 1, 'gamma': 0}

In [175]:
feature_list = separate_model_feature_list
models = [[0, None, None]]
for store in store_data:
    X, Ylog, X_test, Ylog_test, scaler = prepDataframe(store)
    
    # xgboost generalized linear model
    dtrain = xgb.DMatrix(X, label=Ylog)
    dtest  = xgb.DMatrix(X_test,  label=Ylog_test)

    watchlist = [(dtest,'eval'), (dtrain,'train')]
    num_round=1000
    bst = xgb.train(param, dtrain, num_round, watchlist, early_stopping_rounds=150, verbose_eval=False)
#    preds = np.exp(bst.predict(dtest))
#    labels = np.exp(dtest.get_label())
    
    models.append([store.Store, bst, scaler])


Stopping. Best iteration:
[62]	eval-rmse:0.100585	train-rmse:0.003324

Stopping. Best iteration:
[62]	eval-rmse:0.128732	train-rmse:0.003379

Stopping. Best iteration:
[63]	eval-rmse:0.129469	train-rmse:0.003605

Stopping. Best iteration:
[68]	eval-rmse:0.093794	train-rmse:0.003027

Stopping. Best iteration:
[64]	eval-rmse:0.116190	train-rmse:0.003275

Stopping. Best iteration:
[63]	eval-rmse:0.093321	train-rmse:0.003415

Stopping. Best iteration:
[65]	eval-rmse:0.116999	train-rmse:0.003414

Stopping. Best iteration:
[66]	eval-rmse:0.096553	train-rmse:0.003806

Stopping. Best iteration:
[62]	eval-rmse:0.117322	train-rmse:0.003645

Stopping. Best iteration:
[64]	eval-rmse:0.078605	train-rmse:0.003443

Stopping. Best iteration:
[69]	eval-rmse:0.105746	train-rmse:0.003426

Stopping. Best iteration:
[67]	eval-rmse:0.097653	train-rmse:0.003565

Stopping. Best iteration:
[72]	eval-rmse:0.140741	train-rmse:0.003627

Stopping. Best iteration:
[63]	eval-rmse:0.100632	train-rmse:0.003301

Stoppi

In [176]:
test_features = all_test[feature_list_3]
test_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 15 columns):
Store                         41088 non-null int64
DayOfWeek                     41088 non-null int64
day                           41088 non-null int64
month                         41088 non-null int64
year                          41088 non-null int64
CompetitionDistance           41088 non-null float64
CompetitionOpenSinceMonths    41088 non-null int64
Promo                         41088 non-null int64
Promo2                        41088 non-null int64
Promo2SinceMonths             41088 non-null int64
SchoolHolidayVal              41088 non-null int8
StateHolidayVal               41088 non-null int8
StoreTypeVal                  41088 non-null int8
AssortmentVal                 41088 non-null int8
PromoIntervalVal              41088 non-null int8
dtypes: float64(1), int64(9), int8(5)
memory usage: 3.6 MB


In [179]:
store_test_features = test_features[test_features.Store == 1]
store_test_features = store_test_features[feature_list]
test_scaled = models[1][2].transform(store_test_features)
print np.exp(models[1][1].predict(xgb.DMatrix(test_scaled))).shape

(48,)


In [181]:
test_preds = np.ndarray(shape=(1116, 48), dtype=np.float64)
for store in test_features.Store:
    store_test_features = test_features[test_features.Store == store]
    store_test_features = store_test_features[feature_list]
    test_scaled = models[store][2].transform(store_test_features)
    test_preds[store] = np.exp( models[store][1].predict(xgb.DMatrix(test_scaled)))



In [187]:
test_preds[2]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [189]:
store_list = test.Store.unique()

outF = open('subxgb9-trees.csv','wb')
fwriter = csv.writer(outF,delimiter=',')
fwriter.writerow(['Id','Sales'])
row_id = 1
for day in range(len(test_preds[1])):
    for store in store_list:
        fwriter.writerow([row_id,int(test_preds[store][day])])
        row_id += 1
outF.close()